In [ ]:
import pandas as pd

df = pd.read_csv("../data/shrine.csv")
df = df[["id", "name", "name_en", "deity", "deity_en", "lat", "lng", "note_summary_en"]]
df.head()

In [ ]:
import folium

m = folium.Map(location=[35.63, 139.72], zoom_start=13)

for _, row in df.iterrows():
    popup = f"<b>{row['name_en']}</b><br>{row['note_summary_en']}"
    folium.Marker([row["lat"], row["lng"]], popup=popup).add_to(m)

m


![Shrine Network Map](../public/images/shrine_map_meguro-river.PNG)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

# ノード追加
for _, row in df.iterrows():
    G.add_node(row["name_en"], lat=row["lat"], lng=row["lng"])

# エッジ追加（共通祭神を属性に含める）
for i in range(len(df)):
    for j in range(i+1, len(df)):
        set_i = set(df["deity_en"][i].split(","))
        set_j = set(df["deity_en"][j].split(","))
        shared_deities = set_i & set_j  # 共通部分

        if shared_deities:
            # エッジに "shared_deities" 属性をつけて追加
            G.add_edge(
                df["name_en"][i],
                df["name_en"][j],
                shared_deities=", ".join(d.strip() for d in shared_deities)
            )

pos = nx.spring_layout(G, k=0.8, seed=42)

plt.figure(figsize=(12, 9))
nx.draw(G, pos, with_labels=True, node_color="skyblue", edge_color="gray")

# エッジにラベル（共通祭神）を追加
edge_labels = nx.get_edge_attributes(G, 'shared_deities')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

plt.title("Shrine Network with Shared Deities on Edges")
plt.show()


![Shrine Network](../public/images/shrine_network_demo_with_deity_en.png)
